In [ ]:
library(TrialEmulation)

# 1. Setup
trial_pp <- trial_sequence(estimand = "PP")

# Create a directory to save files for later inspection
trial_pp_dir <- file.path(tempdir(), "trial_pp")
trial_itt_dir <- file.path(tempdir(), "trial_pp_dir")
if (!dir.exists(trial_pp_dir)) {
  dir.create(trial_pp_dir)
}

# 2. Data preparation
# Load dummy dataset
data("data_censored") 

# Display first few rows
head(data_censored)

# Save dummy dataset to CSV
write.csv(data_censored, file = "data_censored.csv", row.names = FALSE)

# Setting the observational data for the PP trial
trial_pp <- trial-pp |> set_data( 
  data      = data_censored,
  id        = "id",
  period    = "period",
  treatment = "treatment",
  outcome   = "outcome",
  eligible  = "eligible"
)

# Display trial sequence object
trial_pp

# 3. Weight models and censoring
trial_pp <- trial_pp |> set_censor_weight_model(
  censor_event = "censored",
  numerator    = ~x2,
  denominator  = ~ x2 + x1,
  pool_models  = "numerator",
  model_fitter = stats_glm_logit(save_path = file.path(trial_pp_dir, "switch_models"))
)
trial_pp@censor_weights

#3.1 Censoring due to treatment switching
trial_pp <- trial_pp |>
  set_switch_weight_model(
    numerator    = ~ age,
    denominator  = ~ age + x1 + x3,
    model_fitter = stats_glm_logit(save_path = file.path(trial_pp_dir, "switch_models"))
  )
trial_pp@switch_weights

##  - Numerator formula: treatment ~ age 
##  - Denominator formula: treatment ~ age + x1 + x3 
##  - Model fitter type: te_stats_glm_logit 
##  - Weight models not fitted. Use calculate_weights()


# 4. Calculate Weights
trial_pp <- trial-pp |> calculate_weights()

show_weight_models(trial_pp)

# 5. Specify Outcome Model
trial_pp <- set_outcome_model(trial_pp)

trial_pp@expansion

# 6. Expand Trials
trial_pp <- set_expansion_options(
  trial_pp,
  output     = save_to_datatable(),
  chunk_size = 500
)

# 6.1 Create Sequence of Trials Data
trial_pp <- expand_trials(trial_pp)

# 7. Load or Sample from Expanded Data
trial_pp <- load_expanded_data(trial_pp, seed = 1234, p_control = 0.5)

# 8. Fit Marginal Structural Model
trial_pp <- fit_msm(
  trial_pp,
  weight_cols    = c("weight", "sample_weight"),
  modify_weights = function(w) {
    q99 <- quantile(w, probs = 0.99)
    pmin(w, q99)
  }
)

# Model summary 
trial_pp@outcome_model

# 9. Inference
preds <- predict(
  trial_pp,
  newdata       = outcome_data(trial_pp)[trial_period == 1, ],
  predict_times = 0:10,
  type          = "survival"
)

plot(preds$difference$followup_time, preds$difference$survival_diff,
  type = "l", xlab = "Follow up", ylab = "Survival difference")
lines(preds$difference$followup_time, preds$difference$`2.5%`, type = "l", col = "red", lty = 2)
lines(preds$difference$followup_time, preds$difference$`97.5%`, type = "l", col = "red", lty = 2)
